In [5]:
import tensorflow as tf

class Weighted_sigmoid(tf.keras.losses.Loss):
    def __init__(self, alpha=29, max_value=76.6, k=0.1, name="Weighted_sigmoid", reduction=tf.keras.losses.Reduction.AUTO):
        super(Weighted_sigmoid, self).__init__(name=name, reduction=reduction)
        self.alpha = alpha
        self.max_value = max_value
        self.k = k

    def sigmoid_like_weight(self, x):
        t = -(x - self.max_value / 2) * self.k
        sigmoid = 1 / (1 + tf.exp(t))
        return 1 + self.alpha * sigmoid

    def call(self, y_true, y_pred):
        y_true_rain = y_true[..., 0:1] 
        y_pred_rain = y_pred[..., 0:1]  

        error = y_true_rain - y_pred_rain  
        sample_weight = self.sigmoid_like_weight(y_true_rain)

        weighted_error = sample_weight * tf.square(error)  
        return tf.reduce_mean(weighted_error, axis=0)

    def get_config(self):
        config = super(Weighted_sigmoid, self).get_config()
        config.update({'alpha': self.alpha, 'max_value': self.max_value, 'k': self.k})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)